# JUICE RPWI HF SID23 (PSSR3 rich): L1a QL -- 2023/10/30

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_math_lib as juice_math
import juice_sid23_lib as juice_sid23

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
unit_mode = 0                           # 0: sum    1: /Hz
cal_mode = 0                            # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(0, cal_mode, cal, p_raw_max, p_raw_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder
data_dir = '/Users/user/0-python/JUICE_data/'        # CDF data folder

# *** Ver.2 ***

# 202310 -- SAMPLE
data_name = 'test-CCSDS/sample/cdf/SID23_20231024-0011.cdf'
# data_name = 'test-CCSDS/sample/cdf/SID23_20231024-0024.cdf'
# data_name = 'test-CCSDS/sample/cdf/SID23_20231024-0034.cdf'


# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid23.juice_getdata_hf_sid23(cdf)

# epoch
date = data.epoch[0]
month = date.strftime('%Y%m')
print(data.epoch[0], "-", data.epoch[-1], "[", month, "]")

# data
data.Eu_i = data.Eu_i * 10**(cf/10)
data.Ev_i = data.Ev_i * 10**(cf/10)
data.Ew_i = data.Ew_i * 10**(cf/10)
data.Eu_q = data.Eu_q * 10**(cf/10)
data.Ev_q = data.Ev_q * 10**(cf/10)
data.Ew_q = data.Ew_q * 10**(cf/10)

n_time0 = data.n_time - 1
n_time = data.Eu_i.shape[0]
n_samp = data.Eu_i.shape[1]
print("data size:", data.Eu_i.shape)

n_block = data.N_block[0]
n_length = data.N_feed[0]*128
freq_center = data.freq_center[0]

In [ ]:
# Mode 
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
print("==> Num-samples:", n_time, "   Num-Block:", n_block, "   Length:", n_length, "  frequency_center:", freq_center, "[kHz]")

# Raw data

In [ ]:
fig = plt.figure(figsize=(12, 16))
ax1 = fig.add_subplot(8, 1, 1)
ax3 = fig.add_subplot(8, 1, 2)
ax5 = fig.add_subplot(8, 1, 3)
ax7 = fig.add_subplot(8, 1, 4)
ax8 = fig.add_subplot(8, 1, 5)
ax2 = fig.add_subplot(8, 1, 6)
ax4 = fig.add_subplot(8, 1, 7)
ax6 = fig.add_subplot(8, 1, 8)

ax1.plot(np.ravel(data.Eu_i[:][:]), '-b', linewidth=.5)
ax1.plot(np.ravel(data.Eu_q[:][:]), ':r', linewidth=.5)
ax2.plot(np.ravel(data.Eu_i[:][:]), '-b', linewidth=.5)
ax2.plot(np.ravel(data.Eu_q[:][:]), ':r', linewidth=.5)
ax3.plot(np.ravel(data.Ev_i[:][:]), '-b', linewidth=.5)
ax3.plot(np.ravel(data.Ev_q[:][:]), ':r', linewidth=.5)
ax4.plot(np.ravel(data.Ev_i[:][:]), '-b', linewidth=.5)
ax4.plot(np.ravel(data.Ev_q[:][:]), ':r', linewidth=.5)
ax5.plot(np.ravel(data.Ew_i[:][:]), '-b', linewidth=.5)
ax5.plot(np.ravel(data.Ew_q[:][:]), ':r', linewidth=.5)
ax6.plot(np.ravel(data.Ew_i[:][:]), '-b', linewidth=.5)
ax6.plot(np.ravel(data.Ew_q[:][:]), ':r', linewidth=.5)
ax7.plot(np.ravel(data.time[:]), linewidth=.5)
ax8.plot(np.ravel(data.epoch[:]), '.')
ax1.set_ylabel('Eu [RAW @ ADC]')
ax2.set_ylabel('Eu (1st 100)')
ax3.set_ylabel('Ev [RAW @ ADC]')
ax4.set_ylabel('Ev (1st 100)')
ax5.set_ylabel('Ew [RAW @ ADC]')
ax6.set_ylabel('Ew (1st 100)')
ax7.set_ylabel('Time')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax8.set_ylabel(date)

# range: X-axis
xlim=[0, len(np.ravel(data.Eu_i[:][:]))]
ax1.set_xlim(xlim)
ax3.set_xlim(xlim)
ax5.set_xlim(xlim)
ax7.set_xlim(xlim)
xlim=[128 * data.N_feed[0] * 0, 128 * data.N_feed[0] * .1]
ax2.set_xlim(xlim)
ax4.set_xlim(xlim)
ax6.set_xlim(xlim)
"""
# range: Y-axis
ylim=[-10**(p_raw_max/.2), 10**(p_raw_max/.2)]
ax1.set_ylim(ylim)
ax2.set_ylim(ylim)
ax3.set_ylim(ylim)
"""

# Plot
fig.show
png_fname = work_dir+data_name+'_raw.png'
# fig.savefig(png_fname)

# Get Spectrum

In [ ]:
# Spec mode:           0: low resolution  1; high resolution  
spec = juice_sid23.hf_sid23_getspec(data, unit_mode)
power_str = juice_cdf.power_label(cal_mode, unit_mode)
#
n_time = spec.EuEu.shape[0]
n_block = spec.EuEu.shape[1]
n_freq = spec.EuEu.shape[2]
f_min = spec.freq[0]
f_max = spec.freq[n_freq-1]
d_freq = spec.freq[1]-spec.freq[0]

print(spec.EuEu.shape, "n_time:", n_time, "n_block:", n_block, "n_freq:", n_freq)
print(f_min, "kHz -", f_max, "kHz  (Center:", data.freq_center[0], "kHz  df:", d_freq, "kHz")

# Spectrum

In [ ]:
fig = plt.figure(figsize=(12, 10))
ax1 = fig.add_subplot(4, 1, 1)
ax4 = fig.add_subplot(4, 1, 2)
ax5 = fig.add_subplot(4, 1, 3)
ax6 = fig.add_subplot(4, 1, 4)

s = spec.EuEu[0][0]
print(spec.freq.shape, s.shape)

ax1.plot(np.ravel(spec.EuEu[0][:][:]), '-r', linewidth=.5)
ax1.plot(np.ravel(spec.EvEv[0][:][:]), '-b', linewidth=.5)
ax1.plot(np.ravel(spec.EwEw[0][:][:]), '-b', linewidth=.5)
#
ax4.plot(spec.freq, np.ravel(spec.EuEu[0][0][:]), '-r', linewidth=.5)
ax5.plot(spec.freq, np.ravel(spec.EvEv[0][0][:]), '-r', linewidth=.5)
ax6.plot(spec.freq, np.ravel(spec.EwEw[0][0][:]), '-r', linewidth=.5)
ax4.plot(spec.freq, np.ravel(spec.EuEu[0][np.int16(n_block/2)][:]), '-g', linewidth=.5)
ax5.plot(spec.freq, np.ravel(spec.EvEv[0][np.int16(n_block/2)][:]), '-g', linewidth=.5)
ax6.plot(spec.freq, np.ravel(spec.EwEw[0][np.int16(n_block/2)][:]), '-g', linewidth=.5)
ax4.plot(spec.freq, np.ravel(spec.EuEu[0][n_block-1][:]), '-b', linewidth=.5)
ax5.plot(spec.freq, np.ravel(spec.EvEv[0][n_block-1][:]), '-b', linewidth=.5)
ax6.plot(spec.freq, np.ravel(spec.EwEw[0][n_block-1][:]), '-b', linewidth=.5)
ax1.set_ylabel('EE [RAW @ ADC]')
ax4.set_ylabel('EuEu [RAW @ ADC]')
ax5.set_ylabel('EvEv [RAW @ ADC]')
ax6.set_ylabel('EwEw [RAW @ ADC]')
ax1.set_yscale('log')
ax4.set_yscale('log')
ax5.set_yscale('log')
ax6.set_yscale('log')

# TMP: no filter
ax4.plot(spec.freq, np.ravel(spec.EuEu0[0][0][:]), ':r', linewidth=.5)
ax5.plot(spec.freq, np.ravel(spec.EvEv0[0][0][:]), ':r', linewidth=.5)
ax6.plot(spec.freq, np.ravel(spec.EwEw0[0][0][:]), ':r', linewidth=.5)
ax4.plot(spec.freq, np.ravel(spec.EuEu0[0][np.int16(n_block/2)][:]), ':g', linewidth=.5)
ax5.plot(spec.freq, np.ravel(spec.EvEv0[0][np.int16(n_block/2)][:]), ':g', linewidth=.5)
ax6.plot(spec.freq, np.ravel(spec.EwEw0[0][np.int16(n_block/2)][:]), ':g', linewidth=.5)
ax4.plot(spec.freq, np.ravel(spec.EuEu0[0][n_block-1][:]), ':b', linewidth=.5)
ax5.plot(spec.freq, np.ravel(spec.EvEv0[0][n_block-1][:]), ':b', linewidth=.5)
ax6.plot(spec.freq, np.ravel(spec.EwEw0[0][n_block-1][:]), ':b', linewidth=.5)

"""
# range: X-axis
xlim=[0, len(np.ravel(data.Eu_i[:][:]))]
ax1.set_xlim(xlim)
ax3.set_xlim(xlim)
ax5.set_xlim(xlim)
ax7.set_xlim(xlim)
xlim=[128 * data.N_feed[0] * 0, 128 * data.N_feed[0] * .1]
ax2.set_xlim(xlim)
ax4.set_xlim(xlim)
ax6.set_xlim(xlim)
"""
# range: Y-axis
ylim=[10**(p_raw_min-4), 10**(p_raw_max)]
ax1.set_ylim(ylim)
ax2.set_ylim(ylim)
ax3.set_ylim(ylim)
ax4.set_ylim(ylim)
ax5.set_ylim(ylim)
ax6.set_ylim(ylim)

# Plot
plt.show()
png_fname = work_dir+data_name+'_spec.png'
# fig.savefig(png_fname)

# Frequency - Time diagram

In [ ]:
p_min0 = p_min - 4
p_max0 = p_max
n_num = n_time * n_block
EuEu_2d = np.ravel(spec.EuEu[0])
EvEv_2d = np.ravel(spec.EvEv[0])
EwEw_2d = np.ravel(spec.EwEw[0])
EuEu_2d = EuEu_2d.reshape(n_num, n_freq).transpose()
EvEv_2d = EvEv_2d.reshape(n_num, n_freq).transpose()
EwEw_2d = EwEw_2d.reshape(n_num, n_freq).transpose()
num_1d = np.arange(n_num)

fig2d = plt.figure(figsize=[12,10])
ax1 = fig2d.add_subplot(3, 1, 1)
ax2 = fig2d.add_subplot(3, 1, 2)
ax3 = fig2d.add_subplot(3, 1, 3)

# Y-axis
ax1.set_ylim(f_min, f_max)
ax2.set_ylim(f_min, f_max)
ax3.set_ylim(f_min, f_max)
ax1.set_ylabel('f [kHz]')
ax2.set_ylabel('f [kHz]')
ax3.set_ylabel('f [kHz]')

# X-axis
ax1.set_title('EuEu')
ax2.set_title('EvEv')
ax3.set_title('EwEw')
date = data.epoch[0]
date = date.strftime('%Y/%m/%d')
ax3.set_xlabel(date)

# Plot
p1 = ax1.pcolormesh(num_1d, spec.freq, EuEu_2d[:], norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p2 = ax2.pcolormesh(num_1d, spec.freq, EvEv_2d[:], norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p3 = ax3.pcolormesh(num_1d, spec.freq, EwEw_2d[:], norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")
pp3.set_label(power_str)

plt.show()
png_fname = work_dir+data_name+'_FT.png'
# fig2d.savefig(png_fname)